# Interactive Codes

### There are 2 sections:
##### 1. Bokeh Graphs for all products in differing categories
##### 2. Personalised Skincare Recommendation System


This is where the magic happens! For a detailed explanation of section 1, please refer to the previous jupyter notebook on github. In this notebook, we fully make use of Bokeh's interactive power, and graph out different combinations of product type and skin type. In section 2, users of the recommendation system can select a certain product and find other similar products. It is best to know your skin type as well, but we have included an option if you are unsure of your skin type too!

Do run the code in order for the system to function! It may take a while as we have re-run the UMAP function.

## Section 1: Bokeh Graph

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set()

# importing the data

cosmeticsdata = pd.read_csv('cosmetics.csv')

In [2]:
# installing UMAP

!pip install umap-learn

import umap

import numba

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

from bokeh.io import show, curdoc, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Select, Paragraph, TextInput
from bokeh.layouts import widgetbox, column, row
from ipywidgets import interact 

In [4]:
cosmeticsdata = cosmeticsdata[cosmeticsdata["Ingredients"].str.contains("Visit") == False]
cosmeticsdata = cosmeticsdata[cosmeticsdata["Ingredients"].str.contains("No Info") == False]
cosmeticsdata = cosmeticsdata[cosmeticsdata["Ingredients"].str.contains("NAME") == False]
cosmeticsdata = cosmeticsdata[cosmeticsdata["Ingredients"].str.contains("product package") == False]

cosmeticsdata = cosmeticsdata.set_index('Name')

In [5]:
moisturizer = cosmeticsdata[cosmeticsdata["Label"].str.contains("Moisturizer") == True]
facemask = cosmeticsdata[cosmeticsdata["Label"].str.contains("Face Mask") == True]
cleanser = cosmeticsdata[cosmeticsdata["Label"].str.contains("Cleanser") == True]
treatment = cosmeticsdata[cosmeticsdata["Label"].str.contains("Treatment") == True]
eyecream = cosmeticsdata[cosmeticsdata["Label"].str.contains("Eye cream") == True]
sunprotect = cosmeticsdata[cosmeticsdata["Label"].str.contains("Sun protect") == True]

In [6]:
option_1 = cosmeticsdata.Label.unique().tolist()
option_2 = cosmeticsdata.columns[5:].tolist()

##### This function is to calculate the x and y values of UMAP for each category of product for the plotting of Bokeh graphs later.

In [7]:
def recommendation(op1, op2):
    dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
    dataset1 = dataset1.reset_index()
    
    index = 0
    ingredient_dict = {}
    corpus = []

    for i in range(len(dataset1)):
        ingredients = dataset1['Ingredients'][i]
        ingredients_lower = ingredients.lower()        # change all to lower case
        tokens = ingredients_lower.split(', ')         # split up the ingredients from the string
        corpus.append(tokens)
        for ingredient in tokens:
            if ingredient not in ingredient_dict:      # prevents duplication
                ingredient_dict[ingredient] = index
                index += 1
            
    # maxtrix that is filled with binary values 
    # to check if ingredient is present or absent
    # if present, it will be 1
    # if absent, it will be 0

    M = len(dataset1)           # number of products
    N = len(ingredient_dict)    # number of ingredients
    
    # initialise matrix with 0s
    matrix = np.zeros(shape = (M, N))
    
    
    def one_hot_encoder(tokens):
        x = np.zeros(N)
        for ingredient in tokens:
            index = ingredient_dict[ingredient]
            x[index] = 1
        return x
    
    # make a DTM
    i = 0
    for tokens in corpus:
        matrix[i, :] = one_hot_encoder(tokens)
        i += 1
        
    # dimension reduction with UMAP

    umap_data = umap.UMAP(n_components = 2, min_dist = 0.7, n_neighbors = 5, random_state = 1).fit_transform(matrix)

    # adding 2 new columns X and Y to the dataset

    dataset1['X'] = umap_data[:, 0]
    dataset1['Y'] = umap_data[:, 1]

    return dataset1

##### Create a new CSV file. This file contains more rows of data as each product may cater to different skin types and we are only comparing products within the same category to obtain UMAP values. Hence, multiple rows may exist for each product, containing different sets of X and Y values. 

##### For example, product X is suitable for combination and oily skin. 2 rows will be created for product X. One row contains the X and Y values for product X when compared to other products for combination skin and the other row contains X and Y values for the same product when compared to other products for oily skin.

##### Hence, this new CSV file will contain more rows of data compared to the original dataset "cosmetics.csv". 

##### Since there are 5 categories of skintype and  6 product types, there will be a total of 30 unique labels. For example, Cleanser_Oily, Cleanser_Combination.

In [8]:
dataset_all = pd.DataFrame()

for op1 in option_1:
    for op2 in option_2:
        temp = recommendation(op1, op2)
        temp['Label'] = op1 + '_' + op2
        dataset_all = pd.concat([dataset_all, temp])
        
dataset_all.to_csv('newData.csv', encoding = 'utf-8-sig', index = False)

<ipython-input-7-65688a8ff4ff>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
<ipython-input-7-65688a8ff4ff>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
<ipython-input-7-65688a8ff4ff>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
<ipython-input-7-65688a8ff4ff>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
<ipython-input-7-65688a8ff4ff>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dataset1 = cosmeticsdata[cosmeticsdata["Label"] == op1][cosmeticsdata[op2] == 1]
<ipython-input-7-65688a8ff4ff>:2: U

In [9]:
ds = pd.read_csv('newData.csv')
ds

,Name,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
0,Crème de la Mer,Moisturizer_Combination,LA MER,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1,-36.783270,-0.657504
1,Facial Treatment Essence,Moisturizer_Combination,SK-II,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1,-34.058930,-6.004477
2,Protini™ Polypeptide Cream,Moisturizer_Combination,DRUNK ELEPHANT,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0,-33.301850,-4.885534
3,The Moisturizing Soft Cream,Moisturizer_Combination,LA MER,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1,-21.033564,-6.323969
4,Your Skin But Better™ CC+™ Cream with SPF 50+,Moisturizer_Combination,IT COSMETICS,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1,-30.736635,-1.904555
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,Prime & Protect Mattifying Primer with Broad S...,Sun protect_Sensitive,CANE + AUSTIN,56,4.2,"Ascorbic Acid, Biotin, Camellia Sinensis (Gree...",1,1,1,1,1,10.137806,-6.583157
4093,Quick Dry Body Spray With SolSci-X™ Broad Spec...,Sun protect_Sensitive,MDSOLARSCIENCES,20,3.9,"SD Alcohol 40, Butyloctyl Salicylate, Polyeste...",1,1,1,1,1,8.814378,-7.859425
4094,Sport Continuous Spray SPF 30 - Unscented,Sun protect_Sensitive,COOLA,32,5.0,"Alcohol (Organic), Algae Extract (Organic), Al...",1,1,1,1,1,11.213762,-9.818619
4095,Anti-Aging Armour™ Super Smart Skin-Perfecting...,Sun protect_Sensitive,IT COSMETICS,38,4.1,"Water, Cyclopentasiloxane, Butyloctyl Salicyla...",1,1,1,1,1,11.813240,-4.390383


In [10]:
ds.Label.unique()

array(['Moisturizer_Combination', 'Moisturizer_Dry', 'Moisturizer_Normal',
       'Moisturizer_Oily', 'Moisturizer_Sensitive',
       'Cleanser_Combination', 'Cleanser_Dry', 'Cleanser_Normal',
       'Cleanser_Oily', 'Cleanser_Sensitive', 'Treatment_Combination',
       'Treatment_Dry', 'Treatment_Normal', 'Treatment_Oily',
       'Treatment_Sensitive', 'Face Mask_Combination', 'Face Mask_Dry',
       'Face Mask_Normal', 'Face Mask_Oily', 'Face Mask_Sensitive',
       'Eye cream_Combination', 'Eye cream_Dry', 'Eye cream_Normal',
       'Eye cream_Oily', 'Eye cream_Sensitive', 'Sun protect_Combination',
       'Sun protect_Dry', 'Sun protect_Normal', 'Sun protect_Oily',
       'Sun protect_Sensitive'], dtype=object)

In [11]:
# Load Bokeh Visualisation App

output_notebook()

Loading BokehJS ...

In [12]:
# function to filter the data according to label

def newDataset(dataset, option):
    dataFrame = pd.DataFrame()
    dataFrame = ds[ds['Label'] == option]
    return dataFrame

In [13]:
# function to create a Bokeh plot

def make_plot(src):
    p = figure(width = 500, height = 400, 
                  x_axis_label = 'UMAP 1',
                  y_axis_label = 'UMAP 2')

    p.circle(x = 'X', y = 'Y', source = src,
                size = 10, color = "olive", alpha = 0.8)

    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.5

    # adding a hover tool

    hover = HoverTool(tooltips = [('Product', '@Name'),
                                  ('Brand', '@Brand'), 
                                  ('Price', '$@Price'), 
                                  ('Rank', '@Rank')])

    p.add_tools(hover)
    
    push_notebook()  
    show(p, notebook_handle = True)

In [14]:
# define the callback
def update(op1 =option_1[0], op2 = option_2[0]):
    a_b = op1 + '_' + op2
    toplot = newDataset(ds, a_b)
    make_plot(toplot)

# Get Bokeh Graphs for each product category 

In [15]:
# interact the plot with callback 
interact(update, op1 = option_1, op2 = option_2)

interactive(children=(Dropdown(description='op1', options=('Moisturizer', 'Cleanser', 'Treatment', 'Face Mask'…

<function __main__.update(op1='Moisturizer', op2='Combination')>

# Section 2: Personalised Recommendation System
## User input to get recommended products
### Section 2.1: Skin type is known
#### The code below intakes user's choice of product for comparison and his/her skintype. The skintype is needed to get more accurate estimates of similar products.

In [16]:
# import math library
from math import dist

In [17]:
dsCopy = ds.copy()
dsCopy = dsCopy.set_index('Name')

### Run the code below to input your choice of product. 
Check the 'cosmetics.csv' file for the full list of products.

In [18]:
while True:
    choiceProduct = input("Please enter the product you want to get similar products for: ")         
    choiceSkinType = input("Please enter your skin type (Combination/Dry/Oily/Sensitive/Normal): ")

    myItem = dsCopy.loc[[choiceProduct]]
    if myItem[choiceSkinType].values[0]==1:
        break
        
    if myItem[choiceSkinType].values[0]==0:
        print("Please note that this product is not suitable for your skintype. Try again!")
            
            
print("Successfully updated data")       

Please enter the product you want to get similar products for: Purity Made Simple Cleanser
Please enter your skin type (Combination/Dry/Oily/Sensitive/Normal): Combination
Successfully updated data


In [19]:
pdt = cosmeticsdata.loc[[choiceProduct]]

# get product type
pdtType = pdt['Label'].values[0]

fullLabel = pdtType + '_' + choiceSkinType

In [20]:
dataset1 = dsCopy.copy()
dataset1 = dataset1[dataset1['Label'] == fullLabel]

dataset1

,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
Name,,,,,,,,,,,,
T.L.C. Sukari Babyfacial™,Cleanser_Combination,DRUNK ELEPHANT,80,4.5,"Water, Glycolic Acid, Hydroxyethyl Acrylate/So...",1,1,1,1,0,0.099580,-4.152148
T.L.C. Framboos™ Glycolic Night Serum,Cleanser_Combination,DRUNK ELEPHANT,90,4.3,"Water, Glycolic Acid, Butylene Glycol, Glyceri...",1,1,1,1,0,-0.206441,-2.147541
Green Clean Makeup Meltaway Cleansing Balm with Echinacea GreenEnvy™,Cleanser_Combination,FARMACY,34,4.6,"Cetyl Ethylhexanoate, Caprylic/Capric Triglyce...",1,1,1,1,1,4.908966,-4.720349
Purity Made Simple Cleanser,Cleanser_Combination,PHILOSOPHY,24,4.5,"Water, Sodium Lauroamphoacetate, Sodium Tridec...",1,1,1,1,1,-0.032383,-0.917829
The Rice Polish Foaming Enzyme Powder,Cleanser_Combination,TATCHA,65,4.4,"Microcrystalline Cellulose, Oryza Sativa (Rice...",1,1,1,1,1,4.549874,-1.429712
...,...,...,...,...,...,...,...,...,...,...,...,...
Rosa Centifolia™ No.1 Purity Cleansing Balm,Cleanser_Combination,REN CLEAN SKINCARE,32,4.2,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ce...",1,1,1,1,1,2.008253,-2.049911
Blue Herbal Acne Cleanser Treatment,Cleanser_Combination,KIEHL'S SINCE 1851,22,3.5,"Water, Coco-Glucoside, Propylene Glycol, Ammon...",1,0,0,1,0,2.426944,-3.208080
Pore Refining Detox Double Cleanse,Cleanser_Combination,ERNO LASZLO,55,5.0,"Water, Propanediol, Sodium C14-16 Olefin Sulfo...",1,1,1,1,1,3.898748,-2.811092


In [21]:
myItem = myItem[myItem['Label'] == fullLabel]
myItem

,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
Name,,,,,,,,,,,,
Purity Made Simple Cleanser,Cleanser_Combination,PHILOSOPHY,24,4.5,"Water, Sodium Lauroamphoacetate, Sodium Tridec...",1,1,1,1,1,-0.032383,-0.917829


In [22]:
# obtain X and Y values for choice product and skintype

point1 = np.array([myItem['X'], myItem['Y']])

point1

array([[-0.03238258],
       [-0.91782874]])

In [23]:
# create a new column

dataset1['Distance'] = 0.0

In [24]:
# other items

for i in range(len(dataset1)):
    point2 = np.array([dataset1['X'][i], dataset1['Y'][i]])
    dataset1.Distance[i] = dist(point1, point2)

<ipython-input-24-9d137dfa2f42>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset1.Distance[i] = dist(point1, point2)


In [25]:
# sorting data in ascending order

dataset1 = dataset1.sort_values('Distance')
dataset1.head(6)

,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y,Distance
Name,,,,,,,,,,,,,
Purity Made Simple Cleanser,Cleanser_Combination,PHILOSOPHY,24,4.5,"Water, Sodium Lauroamphoacetate, Sodium Tridec...",1,1,1,1,1,-0.032383,-0.917829,0.000000
Stay Balanced™ Oil Control Cleansing Cloths,Cleanser_Combination,OLEHENRIKSEN,8,4.5,"Water, Hamamelis Virginiana (Witch Hazel) Wate...",1,0,0,1,0,-0.119182,-1.188630,0.284372
Alpha Beta® Pore Perfecting Cleansing Gel,Cleanser_Combination,DR. DENNIS GROSS SKINCARE,38,4.6,"Water, Glycerin, Sodium Lauroyl Methyl Isethio...",1,1,1,1,1,0.341612,-1.142170,0.436120
8% Glycolic Solutions Toner,Cleanser_Combination,PETER THOMAS ROTH,40,4.3,"Water, Glycolic Acid, Alcohol Denat., Methyl G...",1,1,1,1,1,-0.306905,-1.458793,0.606634
A Glowing Regimen Trial Set,Cleanser_Combination,PHILOSOPHY,34,4.8,"Purity Made Simple Cleanser: Water, Sodium Lau...",1,1,1,1,1,-0.806882,-1.206391,0.826509
3% Glycolic Solutions Cleanser,Cleanser_Combination,PETER THOMAS ROTH,38,4.4,"Water, Sodium Laureth Sulfate, Propylene Glyco...",1,1,1,1,1,0.385867,-1.814362,0.989295


### Section 2.2: Skin type is unknown

##### If the user does not know his/her skin type, we can get the UMAP values for products of all categories and plot a bokeh graph with these x and y values. This will give an estimate of the 5 most similar products, regardless of the user's skin type.

##### We need to obtain UMAP values for each product compared to every other product.

In [26]:
CD = cosmeticsdata.copy()

In [27]:
# tokenisation of the ingredients list 

index = 0
ingredient_dict = {}
corpus = []

for i in range(len(cosmeticsdata)):
    ingredients = CD['Ingredients'][i]
    ingredients_lower = ingredients.lower()        # change all to lower case
    tokens = ingredients_lower.split(', ')         # split up the ingredients from the string
    corpus.append(tokens)
    
    for ingredient in tokens:
        if ingredient not in ingredient_dict:      # prevents duplication
            ingredient_dict[ingredient] = index
            index += 1
  


# matrix that is filled with binary values 
# to check if ingredient is present or absent
# if present, it will be 1
# if absent, it will be 0
 
M = len(CD)           # number of products
N = len(ingredient_dict)    # number of ingredients

# initialise matrix with 0s

matrix1 = np.zeros(shape = (M, N))

# filling up the matrix

def one_hot_encoder(tokens):
    x = np.zeros(N)
    
    for ingredient in tokens:
        index = ingredient_dict[ingredient]
        x[index] = 1
    return x

i = 0

for tokens in corpus:
    matrix1[i, :] = one_hot_encoder(tokens)
    i += 1

#### Since this is comparing the product with all other products, we should preserve the global structure and set n_neighbors to a number closer to the maximum number of data (1284). This is to avoid too much focus on the finer details, which can make the results biased. 

##### For example, setting n_neighbours to 5 (too small a number) will cause the 5 recommended results to be products of the same brand as the choice product.

In [28]:
# dimension reduction with UMAP

umap_data = umap.UMAP(n_components = 2, min_dist = 0.7, n_neighbors = 1000, random_state = 1).fit_transform(matrix1)

# adding 2 new columns X and Y to the dataset

CD['X'] = umap_data[:, 0]
CD['Y'] = umap_data[:, 1]

CD

,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y
Name,,,,,,,,,,,,
Crème de la Mer,Moisturizer,LA MER,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1,100.122452,-24.029751
Facial Treatment Essence,Moisturizer,SK-II,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1,105.558647,-25.739443
Protini™ Polypeptide Cream,Moisturizer,DRUNK ELEPHANT,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0,107.661148,-23.679094
The Moisturizing Soft Cream,Moisturizer,LA MER,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1,100.912407,-25.774166
Your Skin But Better™ CC+™ Cream with SPF 50+,Moisturizer,IT COSMETICS,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1,102.269592,-27.337883
...,...,...,...,...,...,...,...,...,...,...,...,...
Daily Deflector™ Moisturizer Broad Spectrum SPF 50+,Sun protect,KATE SOMERVILLE,48,3.9,"Water, Butylene Glycol, PEG-8, Glycerin, Undec...",0,0,0,0,0,105.120941,-27.022434
Yoghurt Nourishing Fluid Veil Face Sunscreen Broad Spectrum SPF 30,Sun protect,KORRES,35,3.9,"Water, Alcohol Denat., Potassium Cetyl Phospha...",1,1,1,1,1,108.706017,-24.069988
Daily Deflector™ Waterlight Broad Spectrum SPF 50+ PA+++ Anti-Aging Sunscreen,Sun protect,KATE SOMERVILLE,48,3.6,"Water, Isododecane, Dimethicone, Butyloctyl Sa...",0,0,0,0,0,105.001213,-27.528728


In [29]:
CD['Distance'] = 0.0

In [30]:
myItem2 = CD.loc[[choiceProduct]]

point1 = np.array([myItem2['X'], myItem2['Y']])
point1

array([[105.41346 ],
       [-30.305857]], dtype=float32)

In [31]:
# other items

for i in range(len(CD)):
    point2 = np.array([CD['X'][i], CD['Y'][i]])
    CD.Distance[i] = dist(point1, point2)

<ipython-input-31-6f842e3f8e3d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CD.Distance[i] = dist(point1, point2)


In [32]:
# sorting data in ascending order

CD = CD.sort_values('Distance')
CD.head(6)

,Label,Brand,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive,X,Y,Distance
Name,,,,,,,,,,,,,
Purity Made Simple Cleanser,Cleanser,PHILOSOPHY,24,4.5,"Water, Sodium Lauroamphoacetate, Sodium Tridec...",1,1,1,1,1,105.413460,-30.305857,0.000000
Clinique Smart Custom-Repair Serum,Treatment,CLINIQUE,60,4.0,"Water , Isododecane , Dimethicone , Cyclopenta...",1,1,1,1,1,105.763672,-30.150152,0.383265
Perfectionist Pro Rapid Firm + Lift Treatment,Treatment,ESTÉE LAUDER,75,5.0,Perfectionist Pro Rpd Frm+Lift Trt Division: E...,1,1,1,1,0,105.096970,-29.975204,0.457708
Purity Made Simple® Facial Cleansing Gel & Eye Makeup Remover,Cleanser,PHILOSOPHY,23,0.0,"Water, Sodium Trideceth Sulfate, Disodium Laur...",0,0,0,0,0,105.954903,-30.292883,0.541598
Clinique Smart Custom-Repair Eye Treatment,Eye cream,CLINIQUE,50,3.7,"Water , Isododecane , Dimethicone , Cyclopenta...",0,0,0,0,0,105.551170,-29.742168,0.580266
Baby Pekee Bar™ + Juju Bar Travel Duo,Cleanser,DRUNK ELEPHANT,18,4.5,-Heilmoor Clay: Tones and detoxifies the skin....,1,1,1,1,0,104.983391,-29.850033,0.626686
